In [30]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
import logging
import os
import requests as re
import pandas as pd
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from pprint import pprint
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from wordcloud import WordCloud

[nltk_data] Downloading package wordnet to /Users/kevins/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kevins/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
papers = pd.read_csv(os.getcwd().replace('scripts', 'data/papers/final_valid_papers.csv'))



In [3]:
stop_words = stopwords.words('english')


In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [5]:
keys = [
        "e3fa6eb97964b552119ea5fc78093789",
"c7cdff212e1739790e728a85953e58d",
"c515ad531ff5767f6b85a20e2c190ca9",
"ef2e9e7f3bda7235f0c079cb38fd2a9a",
"f61a0ca8673900c9539e5b3228c95a51"]

In [6]:
papers


,journal_id,volume_id,year,link,doi,date,pii,eid,scopus_id,citations,authors,affiliations,open_access
0,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104474,2023-08-01,S0309170823001094,2-s2.0-85161258968,8.516126e+10,0.0,"58307014400, 36008901600, 26635952600",60031268:60031268:60007989,True
1,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104475,2023-08-01,S0309170823001100,2-s2.0-85161636300,8.516164e+10,0.0,"53984251100, 35610240400","60104134:60104134, 60022461",True
2,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104476,2023-08-01,S0309170823001112,2-s2.0-85161667219,8.516167e+10,0.0,25822520800,60025063,True
3,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104477,2023-08-01,S0309170823001124,2-s2.0-85161661769,8.516166e+10,0.0,"57210424267, 36160282300, 7404296426",60018394:60018394:60018394,True
4,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104485,2023-08-01,S0309170823001203,2-s2.0-85161678744,8.516168e+10,0.0,"56708113100, 7202660185",60030447:60007566,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26469,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.09.020,2018-01-01,S1364815217304140,2-s2.0-85032268729,8.503227e+10,142.0,"34571084000, 55188785600, 33767605600, 5721638...","60016437:60012197:60012197:60012197, 60025278:...",True
26470,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.09.010,2018-01-01,S1364815216304005,2-s2.0-85032353211,8.503235e+10,54.0,"37111876100, 15069126500, 57189996027, 3574567...","60013141:60025182:60025182:60025182:60013141, ...",True
26471,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.08.002,2018-01-01,S1364815216309550,2-s2.0-85032684995,8.503268e+10,5.0,"7006615627, 49661600900, 15132591800, 66033150...","60021439:60030486, 109682148:60021439:60021439...",True
26472,28,68,2018,https://www.sciencedirect.com/science/article/...,10.1016/j.envsoft.2017.09.011,2018-01-01,S136481521630843X,2-s2.0-85032736133,8.503274e+10,45.0,"56111768300, 37010006200, 43361846100, 5611302...",60068861:60020071:60068861:60024695:60102538:6...,True


In [15]:
#get the full text
def fullText(i):
    doi = papers.iat[i,4]
#     done = False
#     print(doi)
    try:
        js = re.get(
            f"https://api.elsevier.com/content/article/doi/{doi}",
            headers = {"X-ELS-APIKey":"a06b94f351d042ccec3a28009009547d", "Accept":"application/json"}

,
        )
        
        r = js.json()
        
#         print(r)

        full_text = r['full-text-retrieval-response']['originalText']
#         full_text = re.sub(r'^https?:\/\/.[^\s]+', '', full_text)
    
        
        abstr = r['full-text-retrieval-response']['coredata']['dc:description'][:25]
        
        index = full_text.find(abstr)
        if(index==-1):
            other.append(i)
            return None
        full_text = full_text[index:]
        
        
        full_text = ''.join(full_text.split('References')[:-1])
        
        print(full_text)

        full_text = full_text.split(" ")
        full_text_words = list(sent_to_words(full_text))
        # remove stop words
        full_text_words = remove_stopwords(full_text_words)
        
        return full_text_words
    except Exception as e:
        print(str(e))
        if js.status_code == 404:
            no_data.append(i)
        elif js.status_code ==429:
            print("Used up " +str(e))
        

        return None
    



In [16]:
paper_words =fullText(100)

In this paper, we report on experimental and computational studies investigating the evolution of pore structure and permeability of a microporous carbonate rock during chemical dissolution using information provided from X-ray micro-computed tomography (µ-CT) and mercury intrusion porosimetry (MIP) techniques. We consider a chemical dissolution in a core sample by a nonacidic solution wherein a quasi-uniform modification of pore structure occurred. First, we conduct a comparative analysis on the capabilities and limitations of the µ-CT and MIP methods to quantify the evolution of pore size and pore surface area. In particular, we incorporate micropores into the calculations of the pore size-related parameters to highlight the uncertainties that ignoring microporosity causes. In the second part of the paper, we present predictive results on the fractional changes in the permeability of the rock using the Katz-Thompson (KT) and Kozeny-Carman (KC) models. The predictions are based on two

In [80]:
len(set(buzzwords_in_papers['paper_id']))

26478

In [79]:
for i in list(set(buzzwords_in_papers['paper_id'].tolist())):
    tmp = buzzwords_in_papers[buzzwords_in_papers['paper_id']==i]
    
    if tmp.shape[0] ==9:
        print(i)

16
62
70
240
304
612
670
768
790
927
941
991
994
1046
1079
1178
1207
1209
1288
1395
1413
1439
1495
1534
1558
1565
1588
1600
1654
1680
1704
1709
1963
1989
2031
2043
2067
2122
2149
2172
2180
2212
2261
2290
2334
2420
2448
2634
2643
2655
2682
2756
2832
2888
2919
2927
2937
2939
2964
2993
3022
3071
3196
3244
3273
3278
3290
3334
3366
3490
3525
3569
3631
3639
3713
3722
3826
3850
3873
3912
3938
4020
4036
4045
4046
4388
4409
4413
4419
4653
4785
4915
4942
4958
4989
5007
5037
5113
5118
5207
5380
5424
5556
5628
5639
5686
5885
5903
5925
6012
6037
6078
6081
6099
6105
6150
6164
6197
6217
6234
6291
6365
6402
6464
6488
6508
6604
6690
6733
6773
6802
6820
6840
6908
6960
6982
7084
7092
7100
7143
7193
7384
7402
7408
7443
7595
7611
7617
7636
7670
7692
7723
7724
7757
7789
7834
7840
7841
7881
7914
8147
8244
8266
8330
8375
8427
8435
8500
8546
8623
8659
8736
8745
8812
8922
8977
8984
9026
9057
9063
9081
9122
9125
9150
9232
9237
9256
9259
9395
9418
9473
9528
9540
9591
9592
9598
9633
9679
9689
9701
9706
9732
9796
9

In [11]:
paper_words

[[],
 [],
 ['paper'],
 [],
 ['report'],
 [],
 ['experimental'],
 [],
 ['computational'],
 ['studies'],
 ['investigating'],
 [],
 ['evolution'],
 [],
 ['pore'],
 ['structure'],
 [],
 ['permeability'],
 [],
 [],
 ['microporous'],
 ['carbonate'],
 ['rock'],
 [],
 ['chemical'],
 ['dissolution'],
 ['using'],
 ['information'],
 ['provided'],
 [],
 ['ray'],
 ['micro', 'computed'],
 ['tomography'],
 ['ct'],
 [],
 ['mercury'],
 ['intrusion'],
 ['porosimetry'],
 ['mip'],
 ['techniques'],
 [],
 ['consider'],
 [],
 ['chemical'],
 ['dissolution'],
 [],
 [],
 ['core'],
 ['sample'],
 [],
 [],
 ['nonacidic'],
 ['solution'],
 ['wherein'],
 [],
 ['quasi', 'uniform'],
 ['modification'],
 [],
 ['pore'],
 ['structure'],
 ['occurred'],
 ['first'],
 [],
 ['conduct'],
 [],
 ['comparative'],
 ['analysis'],
 [],
 [],
 ['capabilities'],
 [],
 ['limitations'],
 [],
 [],
 ['ct'],
 [],
 ['mip'],
 ['methods'],
 [],
 ['quantify'],
 [],
 ['evolution'],
 [],
 ['pore'],
 ['size'],
 [],
 ['pore'],
 ['surface'],
 ['area']

In [23]:
id2word = corpora.Dictionary(paper_words)

In [10]:
id2word.

In [24]:
texts = paper_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [26]:
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.069*"surface" + 0.063*"pore" + 0.040*"area" + 0.032*"throat" + '
  '0.025*"volume" + 0.017*"space" + 0.016*"total" + 0.010*"incremental" + '
  '0.010*"method" + 0.010*"found"'),
 (1,
  '0.090*"pore" + 0.054*"sample" + 0.030*"images" + 0.020*"based" + '
  '0.017*"size" + 0.017*"experimental" + 0.016*"surface" + 0.013*"solid" + '
  '0.013*"resolution" + 0.012*"possible"'),
 (2,
  '0.049*"pore" + 0.048*"dissolution" + 0.043*"sample" + 0.037*"using" + '
  '0.029*"al" + 0.023*"rock" + 0.023*"porosity" + 0.022*"mercury" + '
  '0.021*"post" + 0.014*"also"'),
 (3,
  '0.045*"ct" + 0.042*"pore" + 0.041*"mip" + 0.017*"permeability" + 0.016*"et" '
  '+ 0.016*"porosity" + 0.015*"surface" + 0.015*"macropores" + 0.013*"method" '
  '+ 0.012*"flow"'),
 (4,
  '0.047*"mip" + 0.045*"diameter" + 0.041*"pore" + 0.040*"data" + 0.017*"core" '
  '+ 0.015*"based" + 0.015*"sample" + 0.015*"al" + 0.014*"calculations" + '
  '0.014*"dissolution"'),
 (5,
  '0.051*"pore" + 0.050*"area" + 0.046*"mip" + 0.020

In [21]:
pyLDAvis.enable_notebook()

In [27]:
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))

In [31]:
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.173903 -0.029745       1        1  11.332822
7      0.029664 -0.080125       2        1  10.720030
0     -0.088104  0.037941       3        1  10.633977
8      0.128427  0.131394       4        1  10.109808
9      0.168001 -0.015132       5        1  10.071564
4     -0.023036  0.058587       6        1   9.968328
6      0.014013 -0.219598       7        1   9.845047
2     -0.117370  0.092300       8        1   9.709632
5      0.039019  0.057649       9        1   9.437685
3      0.023288 -0.033271      10        1   8.171108, topic_info=            Term        Freq       Total Category  logprob  loglift
22            ct   83.000000   83.000000  Default  30.0000  30.0000
7           pore  278.000000  278.000000  Default  29.0000  29.0000
47       surface   89.000000   89.000000  Default  28.0000  28.0000
14   dissolution  109.000000  109.000000  Default  27.0000  27.0000
26           mip  114.000000  114.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
64       results    3.780911   27.215621  Topic10  -5.1328   0.5307
831         post    3.781673   30.387509  Topic10  -5.1326   0.4207
141       volume    3.781810   31.777575  Topic10  -5.1326   0.3760
48          area    3.783036   91.126376  Topic10  -5.1323  -0.6772
341        pores    3.782256   67.638802  Topic10  -5.1325  -0.3793

[595 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
401       6  0.869746        ability
1304      8  0.535352           able
1304     10  0.267676           able
483       6  0.869745  accessibility
469       4  0.866778     accessible
...     ...       ...            ...
517       3  0.169118             μm
517       6  0.056373             μm
517       7  0.112745             μm
517       8  0.507354             μm
517      10  0.056373             μm

[1071 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 8, 1, 9, 10, 5, 7, 3, 6, 4])

In [32]:
papers.loc[100]
    

journal_id                                                      1
volume_id                                                      10
year                                                         2022
link            https://www.sciencedirect.com/science/article/...
doi                               10.1016/j.advwatres.2022.104301
date                                                   2022-10-01
pii                                             S0309170822001658
eid                                            2-s2.0-85136485812
scopus_id                                           85136485812.0
citations                                                     2.0
authors                                  35146801000, 35614045900
affiliations                          60026914, 60026914:60028333
open_access                                                  True
Name: 100, dtype: object

In [33]:
buzzwords_in_papers = pd.read_csv(os.getcwd().replace('scripts', 'data') + '/full_text/buzzwords_in_papers.csv')



In [37]:
tmp = buzzwords_in_papers[buzzwords_in_papers['paper_id']==100]

In [39]:
tmp[tmp['category']=='AI']

,paper_id,category,sub_word,buzz_id
6028,100,AI,neural network,0
6029,100,AI,machine learning,0
6030,100,AI,machine learning,0
6031,100,AI,machine learning,0
6032,100,AI,machine learning,0
6033,100,AI,regression,0
6034,100,AI,regression,0
6035,100,AI,regression,0
6036,100,AI,regression,0
6037,100,AI,regression,0


In [ ]:
buzzwords_in_papers


In [40]:
curr = 0
prev_id =0

In [ ]:
for i in range(buzzwords_in_papers.shape[0]):
    while 

In [55]:
list(set(buzzwords_in_papers['paper_id'].tolist()))[100]

100

In [43]:
processed = set()

In [ ]:
for i in range(buzzwords_in_papers.shape[0]):
    if buzzwords_in_papers.iat[i,0] not in processed:
        
    
    

In [45]:
to_replace = set(buzzwords_in_papers['paper_id'].tolist())



In [82]:
len(to_replace)

26474

In [47]:
buzzwords_in_papers = buzzwords_in_papers.sort_values('paper_id')

In [54]:
for index in range(len(to_replace)):
    found = False
    for i in range(buzzwords_in_papers.shape[0]):
        if buzzwords_in_papers.iat[i,0] == list(to_replace)[index]:
            found = True
            buzzwords_in_papers.iat[i,0] = index
        if found and buzzwords_in_papers.iat[i,0] != list(to_replace)[index]:
            break

KeyboardInterrupt: 

In [84]:
lots_of_words = pd.DataFrame(columns = ['paper_id', 'buzz_id', 'count'])

In [85]:
threshold = 25

In [86]:
for i in range(26474):
    tmp = buzzwords_in_papers[buzzwords_in_papers['paper_id']==i]
    
    grouped = tmp.groupby('buzz_id').count()
    
#     print(grouped)
    
    for j in range(grouped.shape[0]):
        if grouped.iat[j,0] >=threshold:
            lots_of_words.loc[len(lots_of_words.index)] = [i, grouped.iloc[j].name, grouped.iat[j,0]]

In [74]:
lots_of_words

,paper_id,buzz_id,count
0,1,18,102
1,3,6,65
2,7,6,59
3,9,37,62
4,19,0,99
...,...,...,...
3441,26466,24,51
3442,26468,10,75
3443,26470,0,90
3444,26470,6,83


In [76]:
len(set(lots_of_words['paper_id'].tolist()))

2225